# Running an End-to-End Visibility Scan Without TMC

## 1 Setup

First, get the libraries needed for this notebook.

In [ ]:
import json
import os
from datetime import datetime, timedelta, timezone
from time import sleep

from astropy.time import Time
from tango import DeviceProxy

Now, set the IP for visibilities and the DISH IDs that will be used:

In [ ]:
visibility_ip = "10.165.21.2"
dish_ids = ["SKA001", "SKA036"]

Also load in SKA's EPOCH, our TANGO HOST ip, the config file locations and the tango devices to be used. During normal runs these should not need to be changed.

In [ ]:
TANGO_HOST = "10.164.10.193:10000"
SKA_EPOCH = "1999-12-31T23:59:28Z"

# Config file locations
DATA_DIR = "../../data/mid_telescope"
TMC_DIR = f"{DATA_DIR}/tmc"
INITIAL_PARAM_FILE = f"{DATA_DIR}/cbf/sys_params/initial_system_param.json"
ASSIGN_RESOURCES_FILE = f"{DATA_DIR}/csp/assign_resources.json"
CONFIGURE_SCAN_FILE = f"{TMC_DIR}/configure_scan.json"
DELAY_MODEL_FILE = f"{TMC_DIR}/delay_model.json"
SCAN_CONFIG_FILE = f"{TMC_DIR}/scan.json"

# Device FQDNs for device proxies
csp_controller = "mid-csp/control/0"
csp_server = "mid-csp/subarray/01"
tm_server = "ska_mid/tm_leaf_node/csp_subarray_01"
leaf_node_server = "ska_mid/tm_leaf_node/csp_subarray_01"

## 2 Set up Subarray and Turn on the Controller

With the variables and environment set up, start by using the initial parameter JSON to set up the subarray, giving it the VCCs and K-values for the dishes. Once this data is loaded in, the Controller device is turned on.

In [ ]:
os.environ["TANGO_HOST"] = TANGO_HOST
csp_controller_dp = DeviceProxy(csp_controller)
csp_dp = DeviceProxy(csp_server)

sleep(3)

with open(
    INITIAL_PARAM_FILE,
    "r",
    encoding="utf-8",
) as json_data:
    d = json.load(json_data)
    csp_controller_dp.LoadDishCfg(json.dumps(d))

sleep(3)
print(csp_controller_dp.sourceDishVccConfig)
sleep(3)
csp_controller_dp.commandTimeout = 100
sleep(3)
csp_controller_dp.On([])

# WAIT 100 Seconds

## 3 Assign Resources to the Subarray

In [ ]:
with open(ASSIGN_RESOURCES_FILE, "r", encoding="utf-8") as json_data:
    assign_resources_csp = json.load(json_data)
    assign_resources_csp["dish"]["receptor_ids"] = dish_ids
    print(assign_resources_csp)
    csp_dp.AssignResources(json.dumps(assign_resources_csp))

sleep(3)

print(csp_dp.obsState)

## 4 Configure the Scan

Now, prep to run the scan itself by setting up and loading the JSON that will set up the scan. Once this is loaded, the status of the subarray should go first to the CONFIGURING observation state to the READY observation state.

In [ ]:
with open(CONFIGURE_SCAN_FILE, "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    csp_configure_scan = d["csp"]
    csp_configure_scan["cbf"]["fsp"][0]["output_host"] = [[]]
    csp_configure_scan["cbf"][
        "delay_model_subscription_point"
    ] = "ska_mid/tm_leaf_node/csp_subarray_01/delayModel"
    csp_configure_scan["cbf"]["fsp"][0]["output_host"][0] = [0, visibility_ip]
    csp_configure_scan["cbf"]["fsp"][0]["output_port"] = [[0, 21000, 1]]
    print(csp_configure_scan)
    csp_dp.configure(json.dumps(csp_configure_scan))

sleep(5)
print(csp_dp.obsState)

In [ ]:
print(csp_dp.obsState)

## 5 Set Epoch for Delay Model and Load

The delay model can now be loaded in, after setting the epoch to be the difference between now and SKA's defined epoch time.

In [ ]:
ska_epoch_utc = Time(SKA_EPOCH, scale="utc")
ska_epoch_tai = ska_epoch_utc.unix_tai
start_utc_time = Time(datetime.now(timezone.utc) + timedelta(seconds=5), scale="utc")
target_epoch = start_utc_time.unix_tai - ska_epoch_tai

with open(DELAY_MODEL_FILE, encoding="utf-8") as f:
    delay_model = json.load(f)
delayModelProxy = DeviceProxy(leaf_node_server)
delay_model["start_validity_sec"] = target_epoch
print(json.dumps(delay_model))

tm_dp = DeviceProxy(tm_server)
tm_dp.delayModel = json.dumps(delay_model)

sleep(5)

## 6 Start Scan

Finally, with all devices configured and ready, start the scan by passing the scan JSON in via the scan command to the subarray.

In [ ]:
with open(SCAN_CONFIG_FILE, "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    csp_scan = d["csp"]
    print(csp_scan)
    csp_dp.scan(json.dumps(csp_scan))

sleep(2)
print(csp_dp.obsState)

## 7 End the Scan and Clean Up

Once done scanning, free up the devices by stopping the scan, shutting the devices down and releasing the resources they have been assigned.

In [ ]:
csp_dp.EndScan()

In [ ]:
csp_dp.GoToIdle()
sleep(6)
csp_dp.ReleaseAllResources()

In [ ]:
csp_dp.ObsReset()

In [ ]:
csp_controller_dp.Off([])